In [ ]:
# import standard libs
import numpy as np
import pandas as pd
import importlib
from tqdm import tqdm

In [ ]:
# custom utils
import mt_utils as ut
import parameter_utils as pu

In [ ]:
importlib.reload(ut)

In [ ]:
importlib.reload(pu)

In [ ]:
# claspy libs
from claspy.data_loader import load_tssb_dataset
from claspy.data_loader import load_has_dataset
from claspy.segmentation import BinaryClaSPSegmentation

In [ ]:
tssb_res = pd.read_pickle('results/zwischenergebnisse/all_downsampled_TSSB.pkl')
hasc_res = pd.read_pickle('results/zwischenergebnisse/all_downsampled_HASC.pkl')

In [ ]:
tssb = load_tssb_dataset()
hasc = load_has_dataset()

In [ ]:
algos = ["MinMax","MinMaxLTTB","M4","LTTB","LTD", "EveryNth"]
columns = ['algo', 'compression', 'window_size', 'orig_TS_ID', 'time_series', 'true_cps', 'ds_predictions', 'runtime' ]

In [ ]:
def clasp_segmentation(dataset, original, use_downscaled_w):
    import warnings
    warnings.simplefilter("ignore")
    results = []
    for index, row in tqdm(dataset.iterrows(), total=len(dataset), desc="Processing rows"):
        orig = original.iloc[row.Orig_TS_ID]
        orig_ts = orig.time_series
        if orig_ts.ndim==1:
            time_series = orig_ts[row.DS_TS_index]
        else:
            time_series = ut.get_ds_ts_ndim(orig_ts, row.DS_TS_index)

        ds_window_size = int(pu.get_window_size_multivariate(orig_ts)*row.CF)

        try:
            if use_downscaled_w:
                w, pred_cps, score, runtime = ut.evaluate_clasp(time_series, orig.cps, ds_window_size)
            else:
                w, pred_cps, score, runtime = ut.evaluate_clasp(time_series, orig.cps)
        except:
            w, pred_cps, score, runtime = 0, [], 0, 0
           
        results.append((row.DS_Algo, row.CF, w, row.Orig_TS_ID, time_series, orig.cps, pred_cps, runtime))
    return results

# SuSS window size

## TSSB

In [ ]:
results_clasp_tssb = clasp_segmentation(tssb_res, tssb, False)

In [ ]:
results_clasp_df_tssb = pd.DataFrame(results_clasp_tssb, columns=columns)
results_clasp_df_tssb.to_pickle("results/zwischenergebnisse/clasp_on_downsampled_TS_TSSB_origW.pkl")
results_clasp_df_tssb

## HASC

In [ ]:
results_clasp_hasc = clasp_segmentation(hasc_res, hasc, False)

In [ ]:
results_clasp_df_hasc = pd.DataFrame(results_clasp_hasc, columns=columns)
results_clasp_df_hasc.to_pickle("results/zwischenergebnisse/clasp_on_downsampled_TS_HASC_origW.pkl")
results_clasp_df_hasc

# Downscaled window size

## TSSB

In [ ]:
results_clasp_tssb_dsw = clasp_segmentation(tssb_res, tssb, True)

In [ ]:
results_clasp_tssb_dsw_df = pd.DataFrame(results_clasp_tssb_dsw, columns=columns)
results_clasp_tssb_dsw_df.to_pickle("results/zwischenergebnisse/clasp_on_downsampled_TS_TSSB_dsW.pkl")
results_clasp_tssb_dsw_df

## HASC

In [ ]:
results_clasp_hasc_dsw = clasp_segmentation(hasc_res, hasc, True)

In [ ]:
results_clasp_hasc_dsw_df = pd.DataFrame(results_clasp_hasc_dsw, columns=columns)
results_clasp_hasc_dsw_df.to_pickle("results/zwischenergebnisse/clasp_on_downsampled_TS_HASC_dsW.pkl")
results_clasp_hasc_dsw_df